In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import tensorflow as tf

In [0]:
import pandas as pd
import numpy as np
import json
import keras
import tensorflow as tf
import io
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import Conv1D, Conv2D
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import LabelEncoder
import nltk
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.layers import average
import tensorflow_hub as hub
from keras.layers import Average
from keras.layers import Concatenate
nltk.download('punkt')
from numpy import random
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gc
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
#plt.switch_backend('agg')
%matplotlib inline

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
!pip install keras_metrics
import keras_metrics as km

In [0]:
root_path='/content/drive/My Drive/offenseval/2020'

In [0]:
list_df=os.listdir(root_path+"/chunky")
print(list_df)
print(len(list_df))

['data_0', 'data_1', 'data_2', 'data_3', 'data_4', 'data_5', 'data_6', 'data_7', 'data_8', 'data_9', 'data_10', 'data_11', 'data_12', 'data_13', 'data_14', 'data_15', 'data_16', 'data_17', 'data_18', 'data_19', 'data_20', 'data_21', 'data_23', 'data_24', 'data_25', 'data_26', 'data_27', 'data_28', 'data_29', 'data_30', 'data_31', 'data_32', 'data_33', 'data_34', 'data_35', 'data_36', 'data_37', 'data_38', 'data_39', 'data_40', 'data_41', 'data_42', 'data_43', 'data_50', 'data_44', 'data_45', 'data_51', 'data_52', 'data_46', 'data_53', 'data_47', 'data_54', 'data_48', 'data_55', 'data_49', 'data_56', 'data_57', 'data_100', 'data_101', 'data_102', 'data_103', 'data_104', 'data_105', 'data_107', 'data_108', 'data_109', 'data_110', 'data_111', 'data_112', 'data_113', 'data_114', 'data_115', 'data_116', 'data_117', 'data_118', 'data_119', 'data_59', 'data_60', 'data_61', 'data_62', 'data_63', 'data_64', 'data_65', 'data_66', 'data_67', 'data_68', 'data_69', 'data_70', 'data_71', 'data_72', 

In [0]:
df_dummy=pd.read_csv(root_path+"/chunky/data_0")

In [0]:
for file_name in list_df:
  if file_name != 'data_0':
    df_dummy=df_dummy.append(pd.read_csv(root_path+"/chunky/"+file_name),ignore_index=True)

In [0]:
df_dummy.head()

,text,label
0,@user too much thoughts inside his headdd we c...,0.305954
1,"first time i heard his name in camp, he seems ...",0.194293
2,when i go to drink with tsubaki he would alway...,0.295330
3,@user his ass need to stay up :face_with_tears...,0.833349
4,most important tweet of the day : fuck donald ...,0.564527


In [0]:
# from zipfile import ZipFile


In [0]:
# file_name = root_path+"/task_a_distant.zip"

In [0]:
# with ZipFile(file_name,'r') as zip:
#   zip.printdir()
#   zip.extractall(root_path,pwd=b'sem2020-t12')

In [0]:
#train_data=pd.read_csv('/content/drive/My Drive/offenseval/2020/task_a_distant.tsv',delimiter='\t')

In [0]:
#labels=(data['average']>0.5).astype(int)
#print(labels)

In [0]:
# df_chunky=np.array_split(train_data,120)

In [0]:
# max_1=0
# for i in range(120):
#   max_1=max(df_chunky[1].shape[0],max_1)


In [0]:
# import gc

In [0]:
# del(train_data)
# gc.collect()

125

## Twitter embeddings

In [0]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Sentimix/word2vec_twitter_tokens.bin', binary=True,unicode_errors='ignore')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model['fuck'].shape

(400,)

## Cleaning

In [0]:
# !pip install tqdm
# from tqdm import tqdm

In [0]:
#data.reset_index(drop=True,inplace=True)

In [0]:
def remove_pattern(input_txt, pattern,with_space=False):
    r = re.findall(pattern, input_txt)
    if with_space==False:
      for i in r:
        input_txt = re.sub(i, '', input_txt)
    else:
      for i in r:
        input_txt = re.sub(i, ' ', input_txt)
    return input_txt 
!pip install emoji
import emoji
import pickle
import re
with open('/content/drive/My Drive/Sentimix/helper_data/contractions.pkl','rb')as f:
  contractions=pickle.load(f)


from collections import Counter
contractions=Counter(contractions)
with open('/content/drive/My Drive/Sentimix/helper_data/acronyms.pkl','rb')as f:
  acronyms=pickle.load(f)
acronyms=Counter(acronyms)
def acronym(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if acronyms[word]!=0:
        w_l.append(acronyms[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
# with open('/content/drive/My Drive/Sentimix/hinglish_to_english.pickle','rb')as f:
#   hing_to_eng=pickle.load(f)
# hing_to_eng=Counter(hing_to_eng)
def hindi_se_english(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    w_l=[]
    sent=str(df[column][i])
    for word in sent.split():
      if hing_to_eng[word]!=0:
        w_l.append(hing_to_eng[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
# with open('/content/drive/My Drive/Sentimix/Hinglish_utils/Hinglish_Profanity_dict.pkl', 'rb') as handle:
#     cuss_dict=pickle.load(handle)
# cuss_dict=Counter(cuss_dict)
def replace_cuss(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if cuss_dict[word]!=0:
        w_l.append('abuse')
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_contraction(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if contractions[word]!=0:
        w_l.append(contractions[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_pattern_rep(input_txt, pattern,rep_pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
      input_txt = re.sub(i, rep_pattern, input_txt)

    return input_txt 
def cleaning(data_f,cleaning_col,new_col):
  data_f.reset_index(drop=True,inplace=True)
  for i in tqdm(range(data_f.shape[0])):
    data_f[cleaning_col][i]=emoji.demojize(str(data_f[cleaning_col][i]))
  #data_f[cleaning_col]=replace_cuss(data_f,cleaning_col)
  # data_f[new_col]=np.vectorize(remove_pattern)(data_f[cleaning_col],"_",with_space=True)
  # data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],"-",with_space=True)
  # data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],":",with_space=True)
  #data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "@[\w]*","<USR>")
  #data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "http\S+","<URL>")
  data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[cleaning_col], "[0-9]+","<NUM>")
  #data_f[new_col]=hindi_se_english(data_f,cleaning_col)
  data_f[new_col]=remove_contraction(data_f,new_col)
  data_f[new_col]=acronym(data_f,new_col)
  data_f[new_col]=data_f[new_col].str.replace("[^a-zA-Z]<>", " ")
  data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "~",with_space=False)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "!",with_space=True)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], ".",with_space=True)
  #data_f[new_col] = data_f[new_col].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
  return data_f
import numpy as np
#a=cleaning(data,'text','clean_col')


In [0]:
# df_chunky[18].shape

(75612, 4)

In [0]:
# for i in range(96,100):
#   print(i)
#   a=cleaning(df_chunky[i],'text','clean_col')
#   df=pd.DataFrame({'text':a['clean_col'],'label':a['average']})
#   df.to_csv('/content/drive/My Drive/offenseval/2020/chunky/data_'+str(i),index=False)

  0%|          | 0/75611 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  0%|          | 75/75611 [00:00<01:41, 743.78it/s]

96


  0%|          | 131/75611 [00:00<00:57, 1303.44it/s]

97


  0%|          | 135/75611 [00:00<00:56, 1346.38it/s]

98


  0%|          | 132/75611 [00:00<00:57, 1319.07it/s]

99


100%|██████████| 75611/75611 [00:01<00:00, 58537.05it/s]


## Tokenization and Train test Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
df_dummy.shape

(8695339, 2)

In [0]:

df_train,df_test=train_test_split(df_dummy,test_size=0.1)


In [0]:
y_train=(df_train['label']>=0.5).astype(int)
y_test=(df_test['label']>0.5).astype(int)

In [0]:
# df_data=pd.DataFrame({'text':a['clean_col'],'labels':labels})
# df_data.to_csv(root_path+"/clean_eng_task_1.csv",index=False)

In [0]:
!nvidia-smi

Fri Jan 10 10:37:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
max_words =100000
max_len = 30
tok = Tokenizer(max_words)
tok.fit_on_texts(df_dummy['text'].astype(str))


In [0]:
sequences_train = tok.texts_to_sequences(df_train['text'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
sequences_dev = tok.texts_to_sequences(df_test['text'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_dev = sequence.pad_sequences(sequences_dev,maxlen=max_len,padding='post',truncating='post')

In [0]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})


## Attention

In [0]:
embeddings_index = {}
f = open(os.path.join('/content/drive/My Drive/IR_project/glove.6B', 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
word_index=tok.word_index

In [0]:
embedding_matrix_1 = np.zeros([max_words + 1, 300])
for word, i in tok.word_index.items():
    if word in embeddings_index.keys():
      if i>max_words:
        break
      embedding_matrix_1[i] = embeddings_index[word]

In [0]:
from keras.layers import Concatenate
from sklearn.metrics import precision_recall_fscore_support

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.layers import CuDNNGRU,CuDNNLSTM,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.utils import class_weight
class Attention(Layer):
    def __init__(self,step_dim=20,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#max_len=

def BidLstm(maxlen, max_features, embed_size):
    inp1 = Input(shape=(maxlen, ))
    #inp2=Input(shape=(1,))
    #x=Embedding(len(word_index)+1,embed_size)(inp1)
    x1 = Embedding(max_words + 1,embed_size,weights=[embedding_matrix_1],
                  trainable=True)(inp1)
    # x2 = Embedding(len(tok.word_index) + 1,embed_size_2,weights=[embedding_matrix_2],
    #                trainable=True)(inp1)
    # x3 = Embedding(len(tok.word_index) + 1,embed_size_3,weights=[embedding_matrix_3],
    #                trainable=True)(inp1)
    # x1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x1)
    # x2 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x2)
    # x3 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x3)   
    # x1 = Attention(maxlen)(x1)
    # x2 = Attention(maxlen)(x2)
    # x3 = Attention(maxlen)(x3)
    # x=  Concatenate()([x1,x2,x3])
    x1 = CuDNNLSTM(200, return_sequences=True)(x1)   
    x2=  GlobalMaxPooling1D()(x1)
    x3= GlobalAveragePooling1D()(x1)
    x=  Concatenate()([x2,x3])
    x = Dropout(0.1)(x)
    #x = Attention(maxlen)(x)
    # layer = Dense(600,name='FC1')(x)
    # layer = Dense(300,activation='relu')(layer)
    layer = Dense(128,activation='relu')(x)
 #   layer = BatchNormalization(name = 'BN1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.4)(layer)
    layer = Dense(64,name='FC2')(layer)
#    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
   # layer=  Concatenate()([layer,inp2])
    # layer=Dense(256,activation='relu')(layer)
    # layer=Dense(128,activation='relu')(layer)
    layer = Dense(1,name='out_layer',activation='sigmoid')(layer)

    model = Model(inputs=[inp1],outputs=layer)

    return model
model_bi=BidLstm(max_len,max_features=max_words,embed_size=300)

In [0]:
model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])

In [0]:
model_bi.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 30, 300)      30000300    input_2[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)        (None, 30, 200)      401600      embedding_2[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_2 (GlobalM (None, 200)          0           cu_dnnlstm_2[0][0]               
____________________________________________________________________________________________

In [0]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

In [0]:
cp_filepath='/content/drive/My Drive/offenseval/'+'checkpoints/lstm_model_2020a.h5'
cp_check_point=keras.callbacks.ModelCheckpoint(cp_filepath, monitor='val_f1_score', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period=1)
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=2,restore_best_weights=True)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [0]:
model_bi.fit([sequences_matrix_train],y_train,validation_data=([sequences_matrix_dev],y_test),epochs=10,batch_size=1024,class_weight=class_weights,callbacks=[es,cp_check_point])

Train on 7825805 samples, validate on 869534 samples
Epoch 1/10
7825805/7825805 [==============================] - 253s 32us/step - loss: 0.1227 - acc: 0.9501 - f1_score: 0.8599 - val_loss: 0.1015 - val_acc: 0.9584 - val_f1_score: 0.8808
Epoch 2/10
7825805/7825805 [==============================] - 253s 32us/step - loss: 0.0964 - acc: 0.9605 - f1_score: 0.8867 - val_loss: 0.0975 - val_acc: 0.9607 - val_f1_score: 0.8868
Epoch 3/10
7825805/7825805 [==============================] - 253s 32us/step - loss: 0.0838 - acc: 0.9653 - f1_score: 0.8997 - val_loss: 0.0904 - val_acc: 0.9634 - val_f1_score: 0.8933
Epoch 4/10
7825805/7825805 [==============================] - 253s 32us/step - loss: 0.0722 - acc: 0.9702 - f1_score: 0.9130 - val_loss: 0.1014 - val_acc: 0.9621 - val_f1_score: 0.8899
Epoch 5/10
7825805/7825805 [==============================] - 253s 32us/step - loss: 0.0623 - acc: 0.9744 - f1_score: 0.9244 - val_loss: 0.1027 - val_acc: 0.9634 - val_f1_score: 0.8925


In [0]:
y_preds_dl=model_bi.predict(sequences_matrix_dev)

In [0]:
from sklearn.metrics import classification_report

#y_pred = model_bi.predict(X_dev, batch_size=30, verbose=1)

print(classification_report(y_test, y_preds_dl.round()))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98    730838
           1       0.84      0.96      0.89    138696

    accuracy                           0.96    869534
   macro avg       0.91      0.96      0.94    869534
weighted avg       0.97      0.96      0.96    869534

